In [ ]:
import numpy as np
from numpy import *
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from os import listdir
from os.path import isfile, join, isdir

from upsetplot import plot
from upsetplot import from_memberships
from upsetplot import UpSet


In [ ]:
#path = '../results_PU_doublet/'
save_path = '../results_manuscript/doublet_characterization/'
path = '../results_benchmark/'

files = [f.split('_') for f in listdir(path) if ((f[-3:]=='csv') and (f.split('_')[1]!='lib-sze'))]#or (f[-6:]=='_ideal') ))]
files

In [ ]:
methods = np.unique(np.array(files)[:,1])
methods

In [ ]:
data_names = np.sort(np.unique(np.array(files)[:,0]))
data_names

In [ ]:
files = np.array(files)

In [ ]:
width = 15
rows = 3
cols = 6

fig,axes = plt.subplots(figsize=(width,(width/cols)*rows),sharex=True, sharey=True,dpi=300)
d = 1

dfs = []

for data_name in data_names:
    
    print(data_name)

    ano_path  = '../data/mtx_files/' + data_name + '_anno.csv'
    
    #- READ IN BARCODE ANNOTATIONS
    ano = pd.read_csv(ano_path)
    true = pd.factorize(ano.x)[0]
    labels = ano.x
    if (labels[0]=='doublet'):
        tmp = true + 3
        tmp[tmp==3] = 1
        tmp[tmp==4] = 0
        true = tmp

    num_doubs = np.sum(true==1)

    upset_dict = {} 
    [upset_dict.setdefault(i, ['0']) for i in range(sum(true))] 
        
    fs = files[files[:,0]==data_name,:]
    
    
    #keys = np.arange(len(true))
    #dic = dict(zip(keys, [0]*len(keys)))
    for file in fs:
        
        method = file[1]
        
        res = pd.read_csv(path + '_'.join(file))
        scores = res['doublet_scores'] 
                    
        ind = np.argpartition(scores, int(num_doubs)*-1)[int(num_doubs)*-1:]
        
        doublet_call = np.full(len(scores), False)
        doublet_call[ind] = True
        
        doublet_call = doublet_call[true==1]
                                        
        for k in np.arange(len(doublet_call))[doublet_call]:
            upset_dict[k][0] = str(int(upset_dict[k][0]) + 1)
    
    upset_lists=[]
    upset_lists = [categories for categories in upset_dict.values()]
    
    df = pd.DataFrame({'number of methods': np.array(upset_lists).astype(int).flatten()})

    t = np.full(df.shape[0], 'NA      ')
    t[df['number of methods'] == 0] = 'missed'
    t[df['number of methods'] >= 4] = 'captured'
    df['type'] = t
    dfs.append(df)

    ax = plt.subplot(rows, cols, d)
    d+=1
        
    g = sns.histplot(data=df, x="number of methods", hue = 'type', discrete=True, palette="Dark2", hue_order=['captured', 'missed', 'NA      '])
    plt.ylabel('number of doublets')
    
    g.set_xticks(range(9))
    g.set_xticklabels(['0','1','2','3','4','5','6','7','8'])
    g.legend([],[], frameon=False)
    
    x0,x1 = g.get_xlim()
    y0,y1 = g.get_ylim()
    g.set_aspect(abs(x1-x0)/abs(y1-y0))

    plt.title(data_name)

plt.tight_layout()
plt.savefig(save_path + '/histagrams/ALL_hist.png', dpi=300)
plt.show()


In [ ]:
df = pd.concat(dfs)

In [ ]:
MAX_WIDTH = 6.726
fig, ax = plt.subplots(figsize=(MAX_WIDTH,3), dpi=300)

g = sns.histplot(data=df, x="number of methods", hue = 'type', discrete=True, palette="Dark2")
plt.ylabel('number of doublets')

g.set_xticks(range(9))
g.set_xticklabels(['0','1','2','3','4','5','6','7','8'])
g.legend([],[], frameon=False)

#make square
x0,x1 = g.get_xlim()
y0,y1 = g.get_ylim()
g.set_aspect(abs(x1-x0)/abs(y1-y0))

plt.title('Combined')
plt.tight_layout()


plt.savefig(save_path + '/histagrams/combined_hist.png', dpi=300)


In [ ]:
MAX_WIDTH = 6.726
fig, ax = plt.subplots(figsize=(MAX_WIDTH,3), dpi=300)

g = sns.histplot(data=df, x="number of methods", hue = 'type', discrete=True, palette="Dark2")
plt.ylabel('number of doublets')

g.set_xticks(range(9))
g.set_xticklabels(['0','1','2','3','4','5','6','7','8'])

#make square
x0,x1 = g.get_xlim()
y0,y1 = g.get_ylim()

plt.title('Combined')

plt.savefig(save_path + '/histagrams/legend_hist.png', dpi=300)


In [ ]:
#save if it is captured of missed
for data_name in data_names:
    
    print(data_name)

    ano_path  = '../data/mtx_files/' + data_name + '_anno.csv'
    
    #- READ IN BARCODE ANNOTATIONS
    ano = pd.read_csv(ano_path)
    true = pd.factorize(ano.x)[0]
    labels = ano.x
    if (labels[0]=='doublet'):
        tmp = true + 3
        tmp[tmp==3] = 1
        tmp[tmp==4] = 0
        true = tmp

    num_doubs = np.sum(true==1)

    umap_path = '../results_manuscript/UMAPs_preds/UMAP_coords/' + data_name + '_UMAP.npy'
    proj = np.load(umap_path)
        
    fs = files[files[:,0]==data_name,:]
    
    calls = np.zeros(true[true==1].shape)
    for file in fs:
        
        method = file[1]
        
        res = pd.read_csv(path + '_'.join(file))
        scores = res['doublet_scores'] 
                    
        ind = np.argpartition(scores, int(num_doubs)*-1)[int(num_doubs)*-1:]
        
        doublet_call = np.full(len(scores), 0)
        doublet_call[ind] = 1
        
        doublet_call = doublet_call[true==1]
        calls = calls + doublet_call
                

    #0=sing, 1=NA, 2=missed, 3=captured by 4 methods
            
    clls = np.ones(true[true==1].shape)
    clls[calls==0] = 2
    clls[calls>=4] = 3
    true[true==1] = clls
    np.savetxt(save_path + 'data/' + data_name + '_doub_calls.csv', true.astype(int), delimiter=",")

In [ ]:

#files = files[3:4]
d = 0

cols = 4
rows = len(files)

width = 18.267717
width_p_plot = width / cols
height = rows*width_p_plot + rows*0.3

fig,axes = plt.subplots(figsize=(width,height),sharex=True, sharey=True,dpi=300)


for file in files:

    data_name = file[:-19]
    print(data_name)

    enc_file_real = '../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/' + data_name + '_embedding_real.npy'
    enc_file_sim = '../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/' + data_name + '_embedding_sim.npy'
    enc_real = np.load(enc_file_real)
    enc_sim = np.load(enc_file_sim)
    
    num_doubs = enc_sim.shape[0]
    enc = np.vstack([enc_real, enc_sim])
    del(enc_real)
    del(enc_sim)
    
    #read in UMAP coords
    umap_file = save_path + 'UMAP_cords/' + data_name + '_encoding_UMAP.npy'
    if(pl.Path(umap_file).exists()):
        proj = np.load(umap_file)
    else:
        proj = um.UMAP(n_neighbors=7).fit_transform(enc)
        np.save(umap_file, proj)

    ano_path  = '../data/'+ data_dir + '/' + data_name + '_anno.csv'

    #- READ IN BARCODE ANNOTATIONS
    ano = pd.read_csv(ano_path)
    true = pd.factorize(ano.x)[0]
    labels = ano.x
    if (labels[0]=='doublet'):
        tmp = true + 3
        tmp[tmp==3] = 1
        tmp[tmp==4] = 0
        true = tmp
    
    true = np.concatenate([true, np.full(num_doubs, 2)])
    labels = np.concatenate([labels, np.full(num_doubs, 'sim')])
    
    sns.set_style("white")

    # Basic 2D density plot
    d = d+1
    plt.subplot(rows, cols, d)
    plot0 = sns.kdeplot(x=proj[:,0], y=proj[:,1], cmap="crest", shade=True, bw_adjust=.5)
    #axes[0].set_title('Combined')
    plot0.set(yticklabels=[])
    plot0.set(xticklabels=[])
    x0,x1 = plot0.get_xlim()
    y0,y1 = plot0.get_ylim()
    if(d <= 4):
        plt.title('Combined', fontsize=20)
    if((d-1)%4==0):
        plt.ylabel(data_name, fontsize=20)
    #x0,x1 = axes[0].get_xlim()
    #y0,y1 = axes[0].get_ylim()

    d = d+1
    plt.subplot(rows, cols, d)
    plot1 = sns.kdeplot(x=proj[true==0,0], y=proj[true==0,1], cmap="Reds", shade=True, bw_adjust=.5)
    #axes[1].set_title('Singlets')
    plot1.set(yticklabels=[])
    plot1.set(xticklabels=[])
    plot1.set(ylim=(y0, y1), xlim=(x0,x1))
    if(d <= 4):
        plt.title('Real Singlets', fontsize=20)

    d = d+1
    plt.subplot(rows, cols, d)
    plot2 = sns.kdeplot(x=proj[true==1,0], y=proj[true==1,1], cmap="Blues", shade=True, bw_adjust=.5)
    #axes[2].set_title('Doublets')
    plot2.set(yticklabels=[])
    plot2.set(xticklabels=[])
    plot2.set(ylim=(y0, y1), xlim=(x0,x1))
    if(d <= 4):
        plt.title('Real Doublets', fontsize=20)

    d = d+1
    plt.subplot(rows, cols, d)
    plot3 = sns.kdeplot(x=proj[true==2,0], y=proj[true==2,1], cmap="Greens", shade=True, bw_adjust=.5)
    #axes[3].set_title('Simulated')
    plot3.set(yticklabels=[])
    plot3.set(xticklabels=[])
    plot3.set(ylim=(y0, y1), xlim=(x0,x1))
    if(d <= 4):
        plt.title('Simmulated Doublets', fontsize=20)


#plt.suptitle('UMAP of Encoding Densities', fontsize=20)
plt.tight_layout()
plt.savefig(save_path + 'ALL_encoding_UMAP_density.png', dpi=300)
#plt.show()
#plt.close(fig)

In [ ]:
print('hello\nhi')

In [ ]:
d = 0

cols = 5
rows = len(data_names)

width = 18.267717-5
width_p_plot = width / cols
height = rows*width_p_plot + rows*0.3

fig,axes = plt.subplots(figsize=(width,height),sharex=True, sharey=True,dpi=300)

for data_name in data_names:
    
    print(data_name)

    ano_path  = '../data/mtx_files/' + data_name + '_anno.csv'
    
    #- READ IN BARCODE ANNOTATIONS
    ano = pd.read_csv(ano_path)
    true = pd.factorize(ano.x)[0]
    labels = ano.x
    if (labels[0]=='doublet'):
        tmp = true + 3
        tmp[tmp==3] = 1
        tmp[tmp==4] = 0
        true = tmp

    num_doubs = np.sum(true==1)

    umap_path = '../results_manuscript/UMAPs_preds/UMAP_coords/' + data_name + '_UMAP.npy'
    proj = np.load(umap_path)
        
    fs = files[files[:,0]==data_name,:]
    
    #0=sing, 1=NA, 2=missed, 3=captured by 4 methods
    calls = pd.read_csv(save_path + 'data/' + data_name + '_doub_calls.csv', header=None).T.to_numpy()[0]      

    doub_proj = proj[true==1,:]
    
    sns.set_style("white")

    # Basic 2D density plot
    
    d = d+1
    plt.subplot(rows, cols, d)
    plot0 = sns.kdeplot(x=proj[:,0], y=proj[:,1], cmap="crest", shade=True, bw_adjust=.5)
    #axes[0].set_title('All Cells')
    plot0.set(yticklabels=[])
    plot0.set(xticklabels=[])
    x0,x1 = plot0.get_xlim()
    y0,y1 = plot0.get_ylim()
    if(d <= 5):
        plt.title('All Cells', fontsize=14)
    if((d-1)%5==0):
        plt.ylabel(data_name, fontsize=14)

    d = d+1
    plt.subplot(rows, cols, d)
    plot1 = sns.kdeplot(x=proj[true==0,0], y=proj[true==0,1], cmap="Greens", shade=True, bw_adjust=.5)
    plot1.set(yticklabels=[])
    plot1.set(xticklabels=[])
    plot1.set(ylim=(y0, y1), xlim=(x0,x1))
    if(d <= 5):
        plt.title('Experimentally Annotated\nSinglets', fontsize=12)
    
    d = d+1
    plt.subplot(rows, cols, d)
    plot2 = sns.kdeplot(x=proj[true==1,0], y=proj[true==1,1], cmap="Purples", shade=True, bw_adjust=.5)
    plot2.set(yticklabels=[])
    plot2.set(xticklabels=[])
    plot2.set(ylim=(y0, y1), xlim=(x0,x1))
    if(d <= 5):
        plt.title('Experimentally Annotated\nDoublets', fontsize=12)

    d = d+1
    plt.subplot(rows, cols, d)
    plot3 = sns.kdeplot(x=proj[calls==3,0], y=proj[calls==3,1], cmap="Blues", shade=True, bw_adjust=.5)
    plot3.set(yticklabels=[])
    plot3.set(xticklabels=[])
    plot3.set(ylim=(y0, y1), xlim=(x0,x1))
    if(d <= 5):
        plt.title('Captured Doublets', fontsize=14)

    d = d+1
    plt.subplot(rows, cols, d)
    plot4 = sns.kdeplot(x=proj[calls==2,0], y=proj[calls==2,1], cmap="Reds", shade=True, bw_adjust=.5)
    plot4.set(yticklabels=[])
    plot4.set(xticklabels=[])
    plot4.set(ylim=(y0, y1), xlim=(x0,x1))
    if(d <= 5):
        plt.title('Missed Doublets', fontsize=14)


#plt.suptitle(data_name + 'Density UMAP', fontsize=20)
plt.tight_layout()
plt.savefig(save_path + 'density/UMAP_density.png', dpi=300)
plt.show()
plt.close(fig)


In [ ]:
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from scipy.io import mmread
import scipy.sparse as scs
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pathlib as pl


In [ ]:
p = save_path + 'data/df.csv'
if(pl.Path(p).exists()):
    df_keep = pd.read_csv(p)
else:
    #do KNN
    dfs = []
    for data_name in data_names:

        print('-------', data_name, '-------')

        real_path = '../data/mtx_files/' + data_name + '.mtx'

        print('loading in real mtx')
        dat_real = mmread(real_path)
        X = scs.csr_matrix(dat_real).toarray().T

        #Filter genes
        thresh = np.floor(X.shape[0]) * 0.01
        tmp    = np.sum((X>0), axis=0)>thresh
        X = X[:,tmp]

        var = np.var(X, axis=0)
        np.random.seed(3900362577)
        hvgs = np.argpartition(var, -2000)[-2000:]  

        X = X[:,hvgs]

        Y = pd.read_csv(save_path + 'data/' + data_name + '_doub_calls.csv', header=None).T.to_numpy()[0]

        #HYPERPARAMS
        neighbors = 5

        #SCALING
        temp_X = np.log2(X+1)
        np.random.seed(42)
        scaler = StandardScaler().fit(temp_X.T)
        np.random.seed(42)
        temp_X = scaler.transform(temp_X.T).T

        #KNN
        np.random.seed(42)
        pca = PCA(n_components=30)
        pca_proj = pca.fit_transform(temp_X)
        del(temp_X)


        np.random.seed(42)
        knn = KNeighborsClassifier(n_neighbors=neighbors)
        knn.fit(pca_proj,Y)

        preds = knn.predict(pca_proj)
        probs = knn.predict_proba(pca_proj)[:,0]

        #0=sing, 1=NA, 2=missed, 3=captured by 4 methods
        missed_a = np.sum((preds==Y)[Y==2])/np.sum(Y==2)
        captured_a = np.sum((preds==Y)[Y==3])/np.sum(Y==3)

        missed_p = np.mean(probs[Y==2])
        captured_p = np.mean(probs[Y==3])

        df = pd.DataFrame({'knn scores': [missed_a,missed_p,
                                          captured_a,captured_p],
                           'metric': ['accuracy', 'average fraction of singlets', 'accuracy', 'average fraction of singlets'],
                           'which': ['missed', 'missed', 'captured', 'captured']})
        df['data_name'] = data_name

        dfs.append(df)
    df_keep = pd.concat(dfs)
    df_keep.to_csv(save_path + 'data/df.csv')
    df_keep


In [ ]:
df_keep

In [ ]:
df = df_keep.loc[df_keep.metric=='average fraction of singlets']

MAX_WIDTH = 6.726

sns.set(rc={"figure.figsize":(MAX_WIDTH,3)})
sns.set_style("white")

fig, ax1 = plt.subplots(figsize=(MAX_WIDTH,3), dpi=300)
v = sns.violinplot(x='which', y='knn scores', data=df, linewidth=1, edgecolor='black', color='white', ax=ax1, inner=None)# ax2.set_aspect("equal")
#plt.setp(v.collections, alpha=.3)
sns.swarmplot(x='which', y='knn scores', data=df, marker='o', s=7, dodge=False, linewidth=0, hue='data_name', palette=sns.color_palette("hls", 16), ax=ax1)
df_means = df.groupby(['which'])['knn scores'].agg('mean').reset_index()
df_means = df_means.iloc[[1,0],:]
sns.swarmplot(x='which', y='knn scores', data=df_means, marker='o', s=3, dodge=True, linewidth=1, edgecolor='black', color='black', ax=ax1)

plt.legend(bbox_to_anchor=(1,1), loc="upper left", fontsize=6, markerscale=0.75)
plt.title('Doublet Neighborhoods')
plt.ylabel('average fraction of singlets in neighborhood', fontsize=9)
plt.xlabel('')
plt.tick_params(axis='both', which='major', labelsize=10)
#plt.title('Average percentage of singlets in Neighborhood of Doublets')

#make square
x0,x1 = v.get_xlim()
y0,y1 = v.get_ylim()
ax1.set_aspect(abs(x1-x0)/abs(y1-y0))

plt.tight_layout()
plt.savefig(save_path + 'knn.png', dpi=300)

In [ ]:
df = df_keep.loc[df_keep.metric=='accuracy']

MAX_WIDTH = 6.726

sns.set(rc={"figure.figsize":(MAX_WIDTH,3)})
sns.set_style("white")

fig, ax1 = plt.subplots(figsize=(MAX_WIDTH,3), dpi=300)
v = sns.violinplot(x='which', y='knn scores', data=df, linewidth=1, edgecolor='black', color='white', ax=ax1, inner=None)# ax2.set_aspect("equal")
#plt.setp(v.collections, alpha=.3)
sns.swarmplot(x='which', y='knn scores', data=df, marker='o', s=7, dodge=False, linewidth=0, hue='data_name', palette=sns.color_palette("hls", 16), ax=ax1)
df_means = df.groupby(['which'])['knn scores'].agg('mean').reset_index()
df_means = df_means.iloc[[1,0],:]
sns.swarmplot(x='which', y='knn scores', data=df_means, marker='o', s=3, dodge=True, linewidth=1, edgecolor='black', color='black', ax=ax1)

plt.legend(bbox_to_anchor=(1,1), loc="upper left", fontsize=6, markerscale=0.75)
plt.title('Local Label Information')
plt.ylabel('average fraction of singlets in neighborhood', fontsize=9)
plt.xlabel('')
plt.tick_params(axis='both', which='major', labelsize=10)
#plt.title('Average percentage of singlets in Neighborhood of Doublets')
plt.ylabel('average fraction of singlets in the neighborhood')

#make square
x0,x1 = v.get_xlim()
y0,y1 = v.get_ylim()
ax1.set_aspect(abs(x1-x0)/abs(y1-y0))

plt.tight_layout()
plt.savefig(save_path + 'knn_acc.png', dpi=300)